In [27]:
import pandas as pd
import re

base_path = '../data_brut/result'
first_tour_dataframes = []
second_tour_dataframes = []


# Liste des noms de fichiers jusqu'en 2012
file_names_old_format = [
    'cdsp_presi1965t1_circ.csv', 'cdsp_presi1965t2_circ.csv',
    'cdsp_presi1969t1_circ.csv', 'cdsp_presi1969t2_circ.csv',
    'cdsp_presi1974t1_circ.csv', 'cdsp_presi1974t2_circ.csv',
    'cdsp_presi1981t1_circ.csv', 'cdsp_presi1981t2_circ.csv',
    'cdsp_presi1988t1_circ.csv', 'cdsp_presi1988t2_circ.csv',
    'cdsp_presi1995t1_circ.csv', 'cdsp_presi1995t2_circ.csv',
    'cdsp_presi2002t1_circ.csv', 'cdsp_presi2002t2_circ.csv',
    'cdsp_presi2007t1_circ.csv', 'cdsp_presi2007t2_circ.csv',
    'cdsp_presi2012t1_circ.csv', 'cdsp_presi2012t2_circ.csv',
]

# Liste des noms de fichiers de 2017 à 2022
file_names_new_format = [
    'cdsp_presi2017t1_circ.csv', 'cdsp_presi2017t2_circ.csv',
    'cdsp_presi2022t1_circ.csv', 'cdsp_presi2022t2_circ.csv',
]

for file_name in file_names_old_format + file_names_new_format:
    # Construire le chemin complet vers le fichier CSV
    csv_file_path = f"{base_path}/{file_name}"
    
    # Lire le fichier CSV dans un DataFrame
    df = pd.read_csv(csv_file_path, encoding='UTF-8-sig', sep=',')


    # Utiliser une expression régulière pour trouver une année plausible dans le nom de fichier
    match = re.search(r'(\d{4})t\d', file_name)
    if match:
        year = int(match.group(1))  # Convertir l'année trouvée en entier
        df['Année'] = year
    else:
        year = None
    
    if 't1' in file_name:
        tour_dataframes = first_tour_dataframes
    elif 't2' in file_name:
        tour_dataframes = second_tour_dataframes
    else:
        continue


    if file_name in file_names_new_format:
        # Rename columns 
        df.rename(columns={
            'Code du département': 'Code département',
            'Code de la circonscription': 'circonscription',
            'Libellé du département': 'département',
        }, inplace=True)
        # Upper columns Libellé du département


    # Convertir 'Code département' en entier de manière sûre
    df['Code département'] = pd.to_numeric(df['Code département'], errors='coerce').fillna(0).astype(int)
    # Convertir 'circonscription' en entier de manière sûre
    df['circonscription'] = pd.to_numeric(df['circonscription'], errors='coerce').fillna(0).astype(int)
    
    if year < 1986:
        circons_to_include = [1, 2]
    else:
        circons_to_include = [1, 2, 3]

    # Filtrer sur le département 34 et sur les circonscriptions 1 et 2
    if file_name in file_names_old_format:
        df_montpellier = df[(df['Code département'] == 34) & (df['circonscription'].isin(circons_to_include))]
    else:
        df_montpellier = df[(df['Code département'] == 34) & (df['circonscription'].isin(circons_to_include))]

    # Ajouter le DataFrame filtré à la liste
    tour_dataframes.append(df_montpellier)

# Concatenate and save data for each tour
first_tour_data = pd.concat(first_tour_dataframes, ignore_index=True)
second_tour_data = pd.concat(second_tour_dataframes, ignore_index=True)

# regroup data by year
first_tour_data = first_tour_data.groupby('Année').sum().reset_index()
second_tour_data = second_tour_data.groupby('Année').sum().reset_index()


print(first_tour_data.head())  # Preview first tour data
print(second_tour_data.head())  # Preview second tour data

first_tour_data.to_csv('../dataset/first_tour_data.csv', index=False,encoding='utf-8-sig')
second_tour_data.to_csv('../dataset/second_tour_data.csv', index=False,encoding='utf-8-sig')
# to parquet
first_tour_data.to_parquet('../dataset/first_tour_data.parquet', index=False)
second_tour_data.to_parquet('../dataset/second_tour_data.parquet', index=False)




   Année  Code département            département  circonscription  Inscrits  \
0   1965                68         HERAULTHERAULT                3    146031   
1   1969                68         HERAULTHERAULT                3    155779   
2   1974                68         HERAULTHERAULT                3    152386   
3   1981                68         HERAULTHERAULT                3    237560   
4   1988               102  HERAULTHERAULTHERAULT                6    188857   

   Votants  Exprimés  Blancs et nuls  MITTERRAND (CIR)  LECANUET (MRP)  ...  \
0   124249    123242            1007           44685.0         16867.0  ...   
1   118757    117539            1218               0.0             0.0  ...   
2   128752    127713            1039               0.0             0.0  ...   
3   190813    188045            2768               0.0             0.0  ...   
4   152823    150626            2197               0.0             0.0  ...   

   Jean LASSALLE  Philippe POUTOU  FranÃ§ois

In [28]:

# Dictionnaire des candidats et leur orientation politique
candidate_to_orientation = {
    'Extrême gauche': [
        'KRIVINE (LCR)', 'LAGUILLER (LO)', 'BESANCENOT (LCR)', 'ARLETTY (LO)', 'DUCLOS (PCF)', 'BUFFET (PC)',
        'HUE (PCF)', 'LAJOINIE (PCF)', 'MARCHAIS (PCF)', 'MGBADA (AEI)', 'Nathalie ARTHAUD (LO)', 'Philippe POUTOU (NPA)',
        'Jean-Luc MELENCHON (FG)', 'Fabien ROUSSEL'
    ],
    'Gauche': [
        'MITTERRAND (CIR)', 'ROCARD (PSU)', 'MITTERRAND (PS)', 'JOSPIN (PS)', 'HOLLANDE (PS)', 'ROYAL (PS)',
        'Hamon (PS)', 'Anne HIDALGO'
    ],
    'Droite': [
        'DE GAULLE (UNR)', 'POMPIDOU (UDR)', 'CHABAN-DELMAS (UDR)', 'GISCARD D\'ESTAING (RI)', 'CHIRAC (RPR)',
        'BALLADUR (UDF)', 'SARKOZY (UMP)', 'François FILLON', 'Nicolas DUPONT-AIGNAN', 'Valérie PÉCRESSE','LECANUET (MRP)'
    ],
    'Extrême droite': [
        'TIXIER-VIGNANCOUR (EXD)', 'LE PEN (FN)', 'MEGRET (MNR)', 'Marine LE PEN', 'Éric ZEMMOUR'
    ]
}


In [33]:
#chargement du parquet 
df = pd.read_csv('../dataset/first_tour_data.csv',encoding='utf-8-sig')
print(df.head())

# Initialisation des colonnes de regroupement politique
df['Extrême gauche'] = 0
df['Gauche'] = 0
df['Droite'] = 0
df['Extrême droite'] = 0

# Regrouper les résultats des candidats sous les nouvelles colonnes politiques
for orientation, candidates in candidate_to_orientation.items():
    for candidate in candidates:
        if candidate in df.columns:
            df.rename(columns={candidate: orientation}, inplace=True)
            df[orientation] += df[orientation]
            
            
# Sauvegarder le nouveau DataFrame
df.to_csv("../dataset/dataframe_politique.csv", index=False)

   Année  Code département            département  circonscription  Inscrits  \
0   1965                68         HERAULTHERAULT                3    146031   
1   1969                68         HERAULTHERAULT                3    155779   
2   1974                68         HERAULTHERAULT                3    152386   
3   1981                68         HERAULTHERAULT                3    237560   
4   1988               102  HERAULTHERAULTHERAULT                6    188857   

   Votants  Exprimés  Blancs et nuls  MITTERRAND (CIR)  LECANUET (MRP)  ...  \
0   124249    123242            1007           44685.0         16867.0  ...   
1   118757    117539            1218               0.0             0.0  ...   
2   128752    127713            1039               0.0             0.0  ...   
3   190813    188045            2768               0.0             0.0  ...   
4   152823    150626            2197               0.0             0.0  ...   

   Jean LASSALLE  Philippe POUTOU  FranÃ§ois